<a href="https://colab.research.google.com/github/A1171/ColabNotebooks/blob/main/ResNet1D_%D0%B4%D0%BB%D1%8F_%D0%B2%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D1%85_%D1%80%D1%8F%D0%B4%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
#%tensorflow_version 1.x
import keras
from keras import regularizers
from keras import layers
from keras.models import load_model
from matplotlib import pyplot
from sklearn import preprocessing
from sklearn import metrics
import tensorflow as tf
import pickle
import math
from sklearn.preprocessing import LabelBinarizer,StandardScaler,MinMaxScaler
import gc

In [67]:
#Res-net 1D
class DBL(keras.layers.Layer):
    def __init__(self, filters, kernel, strides=1, batch_norm=True,l2Conv=0.0005):
        super(DBL, self).__init__()
        self.filters=filters
        self.kernel_size=kernel
        self.strides=strides
        self.batch_norm=batch_norm
        self.padding='same' if strides == 1 else 'valid'
        self.ZeroPadding1D=layers.ZeroPadding1D(padding=1)
        self.l2Conv=l2Conv
        #print(self.padding)
        self.Conv1D=layers.Conv1D(self.filters,self.kernel_size,strides=self.strides, padding=self.padding,use_bias=not self.batch_norm, kernel_regularizer=regularizers.l2(self.l2Conv))#
        self.BatchNormalization=layers.BatchNormalization(epsilon=0.001)
        self.LeakyReLU=layers.LeakyReLU(alpha=0.1)
    def call(self, inputs):
        if self.strides> 1 :
          x = self.ZeroPadding1D(inputs)
        else:
          x = inputs
        x = self.Conv1D(x)
        if self.batch_norm:
          x = self.BatchNormalization(x)
          x = self.LeakyReLU(x)
        return x
class Res_unit(keras.layers.Layer):
    def __init__(self, filters,l2Conv=0.0005):
        super(Res_unit, self).__init__()
        self.filters=filters
        self.DBL1=DBL(filters // 2, kernel=1,l2Conv=l2Conv)
        self.DBL2=DBL(filters, kernel=3,l2Conv=l2Conv)
        self.add=layers.add
    def call(self, inputs):
      skip_connection = inputs
      x = self.DBL1(inputs)
      x = self.DBL2(x)
      x = self.add([skip_connection , x])
      return x
class ResBlock(keras.layers.Layer):
    def __init__(self, filters, blocks,l2Conv=0.0005):
        super(ResBlock, self).__init__()
        self.filters=filters
        self.blocks=blocks
        self.DBL=DBL(self.filters, kernel=3, strides=2,l2Conv=l2Conv)
        self.ResBlocks=[Res_unit(self.filters,l2Conv=l2Conv) for i in range(self.blocks)]
    def call(self, inputs):
      #print(len(self.ResBlocks))
      x = self.DBL(inputs)
      for i in range(self.blocks):
        #print(x.shape)
        x = self.ResBlocks[i](x)
      return x
def create_ResNet_model(Shape1,Neurons,Filters,l2Conv=0.0005,OutActivation='softmax',OutNeurons=3):
  inputs1 = keras.Input(shape=Shape1)
  lambdaActivity1=0
  lambdaActivity=0
  l2Dense=0.001
  l2ConvB=0.0005
  l2DenseB=0.001
  x = DBL(filters=Filters, kernel=3,l2Conv=l2Conv)(inputs1)
  x = ResBlock(filters=Filters*2, blocks=1,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*4, blocks=2,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*8, blocks=8,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*16, blocks=8,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*32, blocks=4,l2Conv=l2Conv)(x)
  x = layers.Flatten()(x)
  x = layers.Dense(Neurons, activation='tanh',kernel_regularizer=regularizers.l2(l2Dense),bias_regularizer=regularizers.l2(l2Dense),activity_regularizer=regularizers.l1(lambdaActivity1))(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(OutNeurons, activation=OutActivation)(x)
  return keras.Model(inputs=inputs1, outputs=x)
def create_ResNet_short(Shape1,Neurons,Filters,l2Conv=0.0005,OutActivation='softmax',OutNeurons=3):
  inputs1 = keras.Input(shape=Shape1)
  lambdaActivity1=0
  lambdaActivity=0
  l2Dense=0.001
  l2ConvB=0.0005
  l2DenseB=0.001
  x = DBL(filters=Filters, kernel=3,l2Conv=l2Conv)(inputs1)
  x = ResBlock(filters=Filters*2, blocks=1,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*4, blocks=2,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*8, blocks=3,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*16, blocks=3,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*32, blocks=2,l2Conv=l2Conv)(x)
  x = layers.Flatten()(x)
  x = layers.Dense(Neurons, activation='tanh',kernel_regularizer=regularizers.l2(l2Dense),bias_regularizer=regularizers.l2(l2Dense),activity_regularizer=regularizers.l1(lambdaActivity1))(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(OutNeurons, activation=OutActivation)(x)
  return keras.Model(inputs=inputs1, outputs=x)
def create_ResNet_short3In(Shape1,Shape2,Shape3,Neurons,Filters,l2Conv=0.0005,OutActivation='softmax',OutNeurons=3):
  inputs1 = keras.Input(shape=Shape1)
  inputs2 = keras.Input(shape=Shape2)
  inputs3 = keras.Input(shape=Shape3)
  lambdaActivity1=0
  lambdaActivity=0
  l2Dense=0.001
  l2ConvB=0.0005
  l2DenseB=0.001
  x = DBL(filters=Filters, kernel=3,l2Conv=l2Conv)(inputs1)
  x = ResBlock(filters=Filters*2, blocks=1,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*4, blocks=2,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*8, blocks=3,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*16, blocks=3,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*32, blocks=2,l2Conv=l2Conv)(x)
  x = layers.Flatten()(x)
  x1 = DBL(filters=Filters, kernel=3,l2Conv=l2Conv)(inputs2)
  x1 = ResBlock(filters=Filters*2, blocks=1,l2Conv=l2Conv)(x1)
  x1 = ResBlock(filters=Filters*4, blocks=2,l2Conv=l2Conv)(x1)
  x1 = ResBlock(filters=Filters*8, blocks=3,l2Conv=l2Conv)(x1)
  x1 = ResBlock(filters=Filters*16, blocks=3,l2Conv=l2Conv)(x1)
  x1 = ResBlock(filters=Filters*32, blocks=2,l2Conv=l2Conv)(x1)
  x1 = layers.Flatten()(x1)
  x2 = DBL(filters=Filters, kernel=3,l2Conv=l2Conv)(inputs3)
  x2 = ResBlock(filters=Filters*2, blocks=1,l2Conv=l2Conv)(x2)
  x2 = ResBlock(filters=Filters*4, blocks=2,l2Conv=l2Conv)(x2)
  x2 = ResBlock(filters=Filters*8, blocks=3,l2Conv=l2Conv)(x2)
  x2 = ResBlock(filters=Filters*16, blocks=3,l2Conv=l2Conv)(x2)
  x2 = ResBlock(filters=Filters*32, blocks=2,l2Conv=l2Conv)(x2)
  x2 = layers.Flatten()(x2)
  x = layers.Concatenate()([x,x1,x2])
  x = layers.Dense(Neurons, activation='tanh',kernel_regularizer=regularizers.l2(l2Dense),bias_regularizer=regularizers.l2(l2Dense),activity_regularizer=regularizers.l1(lambdaActivity1))(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(OutNeurons, activation=OutActivation)(x)
  return keras.Model(inputs=[inputs1,inputs2,inputs3], outputs=x)

def create_ResNet_Vshort3In(Shape1,Shape2,Shape3,Neurons,Filters,l2Conv=0.0005,OutActivation='softmax',OutNeurons=3):
  inputs1 = keras.Input(shape=Shape1)
  inputs2 = keras.Input(shape=Shape2)
  inputs3 = keras.Input(shape=Shape3)
  lambdaActivity1=0
  lambdaActivity=0
  l2Dense=0.001
  l2ConvB=0.0005
  l2DenseB=0.001
  x = DBL(filters=Filters, kernel=3,l2Conv=l2Conv)(inputs1)
  x = ResBlock(filters=Filters*2, blocks=1,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*4, blocks=1,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*8, blocks=1,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*16, blocks=1,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*32, blocks=1,l2Conv=l2Conv)(x)
  x = layers.Flatten()(x)
  x1 = DBL(filters=Filters, kernel=3,l2Conv=l2Conv)(inputs2)
  x1 = ResBlock(filters=Filters*2, blocks=1,l2Conv=l2Conv)(x1)
  x1 = ResBlock(filters=Filters*4, blocks=1,l2Conv=l2Conv)(x1)
  x1 = ResBlock(filters=Filters*8, blocks=1,l2Conv=l2Conv)(x1)
  x1 = ResBlock(filters=Filters*16, blocks=1,l2Conv=l2Conv)(x1)
  x1 = ResBlock(filters=Filters*32, blocks=1,l2Conv=l2Conv)(x1)
  x1 = layers.Flatten()(x1)
  x2 = DBL(filters=Filters, kernel=3,l2Conv=l2Conv)(inputs3)
  x2 = ResBlock(filters=Filters*2, blocks=1,l2Conv=l2Conv)(x2)
  x2 = ResBlock(filters=Filters*4, blocks=1,l2Conv=l2Conv)(x2)
  x2 = ResBlock(filters=Filters*8, blocks=1,l2Conv=l2Conv)(x2)
  x2 = ResBlock(filters=Filters*16, blocks=1,l2Conv=l2Conv)(x2)
  x2 = ResBlock(filters=Filters*32, blocks=1,l2Conv=l2Conv)(x2)
  x2 = layers.Flatten()(x2)
  x = layers.Concatenate()([x,x1,x2])
  x = layers.Dense(Neurons, activation='tanh',kernel_regularizer=regularizers.l2(l2Dense),bias_regularizer=regularizers.l2(l2Dense),activity_regularizer=regularizers.l1(lambdaActivity1))(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(OutNeurons, activation=OutActivation)(x)
  return keras.Model(inputs=[inputs1,inputs2,inputs3], outputs=x)

def create_ResNet_short4In(Shape1,Shape2,Shape3,Shape4,Neurons,Filters,l2Conv=0.0005,OutActivation='softmax',OutNeurons=3):
  inputs1 = keras.Input(shape=Shape1)
  inputs2 = keras.Input(shape=Shape2)
  inputs3 = keras.Input(shape=Shape3)
  inputs4 = keras.Input(shape=Shape4)
  lambdaActivity1=0
  lambdaActivity=0
  l2Dense=0.001
  l2ConvB=0.0005
  l2DenseB=0.001
  x = DBL(filters=Filters, kernel=3,l2Conv=l2Conv)(inputs1)
  x = ResBlock(filters=Filters*2, blocks=1,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*4, blocks=2,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*8, blocks=3,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*16, blocks=3,l2Conv=l2Conv)(x)
  x = ResBlock(filters=Filters*32, blocks=2,l2Conv=l2Conv)(x)
  x = layers.Flatten()(x)
  x1 = DBL(filters=Filters, kernel=3,l2Conv=l2Conv)(inputs2)
  x1 = ResBlock(filters=Filters*2, blocks=1,l2Conv=l2Conv)(x1)
  x1 = ResBlock(filters=Filters*4, blocks=2,l2Conv=l2Conv)(x1)
  x1 = ResBlock(filters=Filters*8, blocks=3,l2Conv=l2Conv)(x1)
  x1 = ResBlock(filters=Filters*16, blocks=3,l2Conv=l2Conv)(x1)
  x1 = ResBlock(filters=Filters*32, blocks=2,l2Conv=l2Conv)(x1)
  x1 = layers.Flatten()(x1)
  x2 = DBL(filters=int(Filters*0.7), kernel=3,l2Conv=l2Conv)(inputs3)
  x2 = ResBlock(filters=int(Filters*2*0.7), blocks=1,l2Conv=l2Conv)(x2)
  x2 = ResBlock(filters=int(Filters*4*0.7), blocks=2,l2Conv=l2Conv)(x2)
  x2 = ResBlock(filters=int(Filters*8*0.7), blocks=3,l2Conv=l2Conv)(x2)
  x2 = ResBlock(filters=int(Filters*16*0.7), blocks=3,l2Conv=l2Conv)(x2)
  x2 = ResBlock(filters=int(Filters*32*0.7), blocks=2,l2Conv=l2Conv)(x2)
  x2 = layers.Flatten()(x2)
  x3 = DBL(filters=int(Filters*0.5), kernel=3,l2Conv=l2Conv)(inputs4)
  x3 = ResBlock(filters=int(Filters*2*0.5), blocks=1,l2Conv=l2Conv)(x3)
  x3 = ResBlock(filters=int(Filters*4*0.5), blocks=2,l2Conv=l2Conv)(x3)
  x3 = ResBlock(filters=int(Filters*8*0.5), blocks=3,l2Conv=l2Conv)(x3)
  x3 = ResBlock(filters=int(Filters*16*0.5), blocks=3,l2Conv=l2Conv)(x3)
  x3 = ResBlock(filters=int(Filters*32*0.5), blocks=2,l2Conv=l2Conv)(x3)
  x3 = layers.Flatten()(x3)
  x = layers.Concatenate()([x,x1,x2,x3])
  x = layers.Dense(Neurons, activation='tanh',kernel_regularizer=regularizers.l2(l2Dense),bias_regularizer=regularizers.l2(l2Dense),activity_regularizer=regularizers.l1(lambdaActivity1))(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(OutNeurons, activation=OutActivation)(x)
  return keras.Model(inputs=[inputs1,inputs2,inputs3,inputs4], outputs=x)


In [3]:
#Функция оценки результата
def TradeResultMdf(Pred,ProfitArr,Border=[0,0,0]):
  Volume=0
  VolumeBuy=0
  VolumeSell=0
  Profit=0
  WinVolumeBuy=0
  LossVolumeBuy=0
  WinVolumeSell=0
  LossVolumeSell=0
  BuyArr=Pred[:,0]-Border[0]
  SellArr=Pred[:,2]-Border[2]
  BuyArr1=np.logical_and(BuyArr>Pred[:,1],BuyArr>Pred[:,2])
  SellArr1=np.logical_and(SellArr>Pred[:,1],SellArr>Pred[:,0])
  VolumeBuy=np.sum(np.abs(ProfitArr[:,0][BuyArr1]))
  ProfitBuy=np.sum(ProfitArr[:,0][BuyArr1])
  VolumeSell=np.sum(np.abs(ProfitArr[:,1][SellArr1]))
  ProfitSell=np.sum(ProfitArr[:,1][SellArr1])
  VolumeAll=np.sum(np.abs(ProfitArr))/2
  Recall=(VolumeBuy+VolumeSell)/VolumeAll
  del BuyArr
  del SellArr
  del BuyArr1
  del SellArr1
  return ProfitBuy,ProfitSell, VolumeBuy,VolumeSell, Recall
#Функция расчета доходности от границы классификации
def ClassBorderProfit(Pred,ProfitArr,STDBorders,AddSTDCorrection=True):
  NumOuts2=Pred.shape[1]//NClasses
  NumSTDBorders=len(STDBorders)
  PFArr=np.zeros((NumSTDBorders,NumOuts2))
  RecallArr=np.zeros((NumSTDBorders,NumOuts2))
  PFArrBuy=np.zeros((NumSTDBorders,NumOuts2))
  PFArrSell=np.zeros((NumSTDBorders,NumOuts2))
  VolumeBuy=np.zeros((NumSTDBorders,NumOuts2))
  VolumeSell=np.zeros((NumSTDBorders,NumOuts2))
  ProfitBuy=np.zeros((NumSTDBorders,NumOuts2))
  ProfitSell=np.zeros((NumSTDBorders,NumOuts2))
  if(AddSTDCorrection):
    Pred1=(Pred-Pred.mean(axis=0))/Pred.std(axis=0)
  else:
    Pred1=Pred
  for i in range(NumOuts2):
    for j in range(NumSTDBorders):
      ProfitBuy1_1,ProfitSell1_1, VolumeBuy1_1,VolumeSell1_1, Recall1_1=TradeResultMdf(Pred1,ProfitArr,Border=[STDBorders[j],STDBorders[j],STDBorders[j]])
      RecallArr[j,i]=Recall1_1
      VolumeBuy[j,i]=VolumeBuy1_1
      VolumeSell[j,i]=VolumeBuy1_1
      ProfitBuy[j,i]=ProfitBuy1_1
      ProfitSell[j,i]=ProfitSell1_1
      
      if(VolumeBuy1_1+VolumeSell1_1>0):
        PFArr[j,i]=(ProfitBuy1_1+ProfitSell1_1)/(VolumeBuy1_1+VolumeSell1_1)
      else:
        if(j>0):
          PFArr[j,i]=PFArr[j-1,i]
        else:
          PFArr[j,i]=0
      if(VolumeBuy1_1>0):
        PFArrBuy[j,i]=ProfitBuy1_1/VolumeBuy1_1
      else:
        if(j>0):
          PFArrBuy[j,i]=PFArrBuy[j-1,i]
        else:
          PFArrBuy[j,i]=0
      if(VolumeSell1_1>0):
        PFArrSell[j,i]=ProfitSell1_1/VolumeSell1_1
      else:
        if(j>0):
          PFArrSell[j,i]=PFArrSell[j-1,i]
        else:
          PFArrSell[j,i]=0
  del Pred1
  gc.collect()
  return PFArr,RecallArr,PFArrBuy,PFArrSell,VolumeBuy,VolumeSell,ProfitBuy,ProfitSell


Дальше куча процедур для тестирования к Res-net не относится

In [64]:
class MyCustomCallback(keras.callbacks.Callback):
    def __init__(self,STDBorders,ChartTestList,ChartValList,EpochStep=1,BatchStep=1):
      self.STDBorders=STDBorders
      self.ChartTestList=ChartTestList
      self.ChartValList=ChartValList
      self.EpochStep=EpochStep
      self.BatchStep=BatchStep
                 
    def on_epoch_end(self, epoch, logs=None):
      if(epoch%self.EpochStep==0):
          pred2=[]
          pred3=[]
          Profit2=[]
          Profit3=[]
          for i in range(0,test_datagen_est.__len__(),self.BatchStep):
              Batch=test_datagen_est[i]
              if(len(pred2)==0):
                pred2=self.model.predict(Batch[0],verbose=0)
                Profit2=Batch[2]
              else:
                pred2=np.vstack((pred2,self.model.predict(Batch[0],verbose=0)))
                Profit2=np.vstack((Profit2,Batch[2]))
          for i in range(0,val_datagen_est.__len__(),self.BatchStep):
              Batch=val_datagen_est[i]
              if(len(pred3)==0):
                pred3=self.model.predict(Batch[0],verbose=0)
                Profit3=Batch[2]
              else:
                pred3=np.vstack((pred3,self.model.predict(Batch[0],verbose=0)))
                Profit3=np.vstack((Profit3,Batch[2]))
          gc.collect()
          PFArr1,RecallArr1,PFArrBuy1,PFArrSell1,VolumeBuy1,VolumeSell1,ProfitBuy1,ProfitSell1=ClassBorderProfit(pred2,Profit2,self.STDBorders,AddSTDCorrection=True)
          PFArr2,RecallArr2,PFArrBuy2,PFArrSell2,VolumeBuy2,VolumeSell2,ProfitBuy2,ProfitSell2=ClassBorderProfit(pred3,Profit3,self.STDBorders,AddSTDCorrection=True)
          self.ChartTestList.append([PFArr1[:,0],RecallArr1[:,0],(VolumeBuy1+VolumeSell1)[:,0]])
          self.ChartValList.append([PFArr2[:,0],RecallArr2[:,0],(VolumeBuy2+VolumeSell2)[:,0]])
          print()
          print("Check std=",self.STDBorders,"P",[int(PFArr1[Item,0]*1000)/10 for Item in range(len(self.STDBorders))],[int(PFArr2[Item,0]*1000)/10 for Item in range(len(self.STDBorders))],"PM",[int((PFArr1[Item,0]+PFArr2[Item,0])*1000)/10/2 for Item in range(len(self.STDBorders))],"R",[int(RecallArr1[Item,0]*1000)/10 for Item in range(len(self.STDBorders))],[int(RecallArr2[Item,0]*1000)/10 for Item in range(len(self.STDBorders))])
      
custom_objects = {"MyCustomCallback":MyCustomCallback}

In [19]:
def Template51(y_data_train2):#4 периода TP=0.4 SL=0.2
  if(y_data_train2[24]==1 and y_data_train2[4]>-1 and y_data_train2[3]>-1 and y_data_train2[2]>-1):
    return 1
  if(y_data_train2[24]==-1 and y_data_train2[4]<1 and y_data_train2[3]<1 and y_data_train2[2]<1):# and y_data_train2[43]==-1
    return -1
  return 0
def Template51Profit(y_data_train2):#4 периода TP=0.4 SL=0.2
  BuyPr=y_data_train2[34]
  if(BuyPr<-0.5):BuyPr=-0.5
  SellPr=-y_data_train2[34]
  if(SellPr<-0.5):SellPr=-0.5
  return np.array([BuyPr,SellPr])

In [22]:
def GetWeightMatrixFromDatagen(Datagen,Step=5):
  y_in_data_tr2=[]
  for i in range(0,train_datagen.__len__(),5):
    if(len(y_in_data_tr2)==0):
      y_in_data_tr2=train_datagen[i][1]
    else:
      y_in_data_tr2=np.vstack((y_in_data_tr2,train_datagen[i][1]))
  Argmax_y=np.argmax(y_in_data_tr2,axis=1)
  WeightMatrix18List=[]
  #print(y_in_data_tr2.shape,len(Argmax_y[Argmax_y==0]),len(Argmax_y[Argmax_y==1]))
  for i in range(y_in_data_tr2.shape[1]):
    WeightMatrix18List.append(1/len(Argmax_y[Argmax_y==i]))
  WeightMatrix18List=np.array(WeightMatrix18List)
  WeightMatrix18List=WeightMatrix18List/np.amax(WeightMatrix18List)
  Multiplier=np.sum(1/WeightMatrix18List)/len(WeightMatrix18List)
  WeightMatrix18List=WeightMatrix18List*Multiplier
  WeightMatrix2=dict(enumerate(WeightMatrix18List.flatten(), 0))
  del y_in_data_tr2
  del Argmax_y
  gc.collect()
  return WeightMatrix2
import gc
class TMultiTF_Datagen2(tf.keras.utils.Sequence):
    def __init__(self, x_data_tr,y_in_data_tr,FeaturesLists,ScaleGroup,Binalizers,LenSeq,batch_size,TemplateFunction,ProfitFunction,TimeOut=False,UseDOW=False,GenerateProfitArr=False):
        self.x_data_tr=x_data_tr
        self.y_in_data_tr=y_in_data_tr
        self.FeaturesLists=FeaturesLists
        self.ScaleGroup=ScaleGroup
        self.LenSeq=LenSeq
        self.batch_size = batch_size
        self.TimeOut=TimeOut
        self.UseDOW=UseDOW
        self.GenerateProfitArr=GenerateProfitArr
        self.TemplateFunction=TemplateFunction
        self.ProfitFunction=ProfitFunction
        self.FeaturesSet=[]
        self.Scallers=[]
        BinalizerDOW,BinalizerHour,BinalizerMin,BinalizerRSI=Binalizers
        if(not UseDOW):
              if(x_data_tr.shape[0]>0):self.Time_Bin_tr=np.hstack((BinalizerHour.transform(np.array((x_data_tr[:,HourPos]+0.1),dtype=int)),BinalizerMin.transform(np.array((x_data_tr[:,MinPos]+0.1),dtype=int))))
        else:
              if(x_data_tr.shape[0]>0):self.Time_Bin_tr=np.hstack((BinalizerDOW.transform(np.array((x_data_tr[:,DOWPos]+0.1),dtype=int)),BinalizerHour.transform(np.array((x_data_tr[:,HourPos]+0.1),dtype=int)),BinalizerMin.transform(np.array((x_data_tr[:,MinPos]+0.1),dtype=int))))
        for i in range(len(self.FeaturesLists)):
          self.FeaturesSet.append(x_data_tr[:,self.FeaturesLists[i]])
          if(ScaleGroup[i]<2):
            self.Scallers.append([])
          elif(ScaleGroup[i]==2):#minmax
            self.Scallers.append(StandardScaler())
            self.Scallers[-1].fit(self.FeaturesSet[-1])
            self.FeaturesSet[-1]=self.Scallers[-1].transform(self.FeaturesSet[-1])
          elif(ScaleGroup[i]==3):#std
            self.Scallers.append(MinMaxScaler())
            self.Scallers[-1].fit(self.FeaturesSet[-1])
            self.FeaturesSet[-1]=self.Scallers[-1].transform(self.FeaturesSet[-1])

        self.TrTestStart=(self.LenSeq+1)+1
        self.TrainStop=len(self.x_data_tr)
        gc.collect()
        
    def __len__(self):
        #print("__len__",self.TrTestStart,self.TrainStop, int(np.floor((self.TrainStop-self.TrTestStart) / self.batch_size)))
        len1=int(np.floor((self.TrainStop-self.TrTestStart) / self.batch_size))
        if(len1*self.batch_size<self.TrainStop-self.TrTestStart):
          len1+=1
        return len1
        

    def __getitem__(self, idx):
        TrBunchStart=self.TrTestStart+idx*self.batch_size
        TrBunchStop=TrBunchStart+self.batch_size
        if(TrBunchStop>self.TrainStop):TrBunchStop=self.TrainStop
        x_data_train_t1=[]
        x_data_train_time=[]
        y_in_data_train2=[]
        SequencyLenP1=self.LenSeq
        MiddleClass=NClasses//2
        
        for i in range(TrBunchStart+1,TrBunchStop+1,RecordsStep):
          Record=[]
          for j in range(len(self.FeaturesSet)):
            Arr1=self.FeaturesSet[j]
            Record.append(Arr1[i-SequencyLenP1:i].copy())
            if(self.ScaleGroup[j]==1):
              GroupMax=np.amax(Record[-1])
              GroupMin=np.amin(Record[-1])
              GroupStd1=Record[-1].std()
              Div=(GroupMax-GroupMin)
              if(Div<=0):Div=GroupStd1/10
              if(Div<=0):Div=0.0001
              Record[-1]=(Record[-1]-GroupMin)/Div
          Record=np.hstack(Record)
          x_data_train_t1.append(Record)
          x_data_train_time.append(self.Time_Bin_tr[i-1].copy())
          if(type(self.y_in_data_tr)!=type(None)):y_in_data_train2.append(self.y_in_data_tr[i-1])
        x_data_train_t1=np.array(x_data_train_t1)
        x_data_train_time=np.array(x_data_train_time)
        if(type(self.y_in_data_tr)!=type(None)):y_in_data_train2=np.array(y_in_data_train2)
        TemplTr=[]
        TemplTrPr=[]
        if(type(y_in_data_tr)!=type(None)):
          for i in range(y_in_data_train2.shape[0]):
            TemplTr.append(self.TemplateFunction(y_in_data_train2[i]))
            TemplTrPr.append(self.ProfitFunction(y_in_data_train2[i]))
        TemplTr=np.array(TemplTr)
        TemplTrPr=np.array(TemplTrPr)

        y_data_train_tpl=np.zeros((y_in_data_train2.shape[0],NClasses))
        y_data_train_tpl[np.where(TemplTr==1),MiddleClass-1]=1
        y_data_train_tpl[np.where(TemplTr==0),MiddleClass]=1
        y_data_train_tpl[np.where(TemplTr==-1),MiddleClass+1]=1
        if(NClasses>3):
          y_data_train_tpl[np.where(TemplTr==2),MiddleClass-2]=1
          y_data_train_tpl[np.where(TemplTr==-2),MiddleClass+2]=1
        if(NClasses>5):
          y_data_train_tpl[np.where(TemplTr==3),MiddleClass-3]=1
          y_data_train_tpl[np.where(TemplTr==-3),MiddleClass+3]=1
        
        InputsList=[]
        InputsList.append(x_data_train_t1)
        if(self.TimeOut):
          InputsList.append(x_data_train_time)
        

        if(len(InputsList)==1):
          InputsList=InputsList[0]

        gc.collect()
        #print(InputsList[0].shape,InputsList[1].shape,y_data_train_tpl.shape)
        
        if(self.GenerateProfitArr):return (InputsList,y_data_train_tpl,TemplTrPr)
        else:return (InputsList,y_data_train_tpl)


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
SequencyLen=32
RecordsStep=1
NClasses=3

In [12]:
HistoryFileDir="drive/My Drive/NN/DiplShares/"
#Загрузка массива данных и разбивка на выборки
AdditNames=["SBER_M5_Linear5"]
TrainTestGap=SequencyLen+100
trainPart=0.8
testPart=0.1
x_data_1 = np.loadtxt(HistoryFileDir+"DataFileI1_"+AdditNames[0]+'.txt')
y_data_1 = np.loadtxt(HistoryFileDir+"DataFileO1_"+AdditNames[0]+'.txt')
x_data_tr=x_data_1[:int(len(x_data_1)*trainPart)]
x_data_test=x_data_1[int(len(x_data_1)*trainPart+TrainTestGap):int(len(x_data_1)*(trainPart+testPart)+TrainTestGap)]
x_data_val=x_data_1[int(len(x_data_1)*(trainPart+testPart)+TrainTestGap*2):]
y_in_data_tr=y_data_1[:int(len(x_data_1)*trainPart)]
y_in_data_test=y_data_1[int(len(x_data_1)*trainPart+TrainTestGap):int(len(x_data_1)*(trainPart+testPart)+TrainTestGap)]
y_in_data_val=y_data_1[int(len(x_data_1)*(trainPart+testPart)+TrainTestGap*2):]

Биналайзеры для временни

In [15]:
DOWPos=3
HourPos=5
MinPos=6
BinalizerDOW=LabelBinarizer()
BinalizerHour=LabelBinarizer()
BinalizerMin=LabelBinarizer()
BinalizerRSI=LabelBinarizer()
BinalizerDOW.fit(np.array((x_data_tr[:,DOWPos]+0.1),dtype=int))
BinalizerHour.fit(np.array((x_data_tr[:,HourPos]+0.1),dtype=int))
BinalizerMin.fit(np.array((x_data_tr[:,MinPos]+0.1),dtype=int))
Binalizers=[BinalizerDOW,BinalizerHour,BinalizerMin,BinalizerRSI]

In [16]:
adamopt=tf.keras.optimizers.Adam(learning_rate=0.0004, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [66]:
#Создаем группы признаков. Данные будут масштабироваться целеком группами
FeaturesLists=[[8,9,10,11],[12],[15,18,19,21]]
#Выбираем способ масштабирования для каждой из групп
#0 - без масштабирования
#1 - Min/Max масштабирование целиком для группы
#2 - Min/Max масштабирование отдельно для каждого признака
#3 - Std масштабирование отдельно для каждого признака
ScaleGroup=[1,1,2]
#имена отобранных признаков для графиков
Names123=["OM541","HM541","LM541","CM541","VM541","1RSI14","2RSI14","3RSI3","3RSI14"]
BatchSize=4096
SequencyLen=32
GenerateProfitArr=False
train_datagen=TMultiTF_Datagen2(x_data_tr,y_in_data_tr,FeaturesLists,ScaleGroup,Binalizers,SequencyLen,BatchSize,Template51,Template51Profit,TimeOut=False,UseDOW=False,GenerateProfitArr=GenerateProfitArr)
test_datagen=TMultiTF_Datagen2(x_data_test,y_in_data_test,FeaturesLists,ScaleGroup,Binalizers,SequencyLen,BatchSize,Template51,Template51Profit,TimeOut=False,UseDOW=False,GenerateProfitArr=GenerateProfitArr)
val_datagen=TMultiTF_Datagen2(x_data_val,y_in_data_val,FeaturesLists,ScaleGroup,Binalizers,SequencyLen,BatchSize,Template51,Template51Profit,TimeOut=False,UseDOW=False,GenerateProfitArr=GenerateProfitArr)
#datagen for profit check. It have 1 batch with full length and profit estimate arrays
BatchSize=1024
GenerateProfitArr=True
train_datagen_est=TMultiTF_Datagen2(x_data_tr,y_in_data_tr,FeaturesLists,ScaleGroup,Binalizers,SequencyLen,BatchSize,Template51,Template51Profit,TimeOut=False,UseDOW=False,GenerateProfitArr=GenerateProfitArr)
test_datagen_est=TMultiTF_Datagen2(x_data_test,y_in_data_test,FeaturesLists,ScaleGroup,Binalizers,SequencyLen,BatchSize,Template51,Template51Profit,TimeOut=False,UseDOW=False,GenerateProfitArr=GenerateProfitArr)
val_datagen_est=TMultiTF_Datagen2(x_data_val,y_in_data_val,FeaturesLists,ScaleGroup,Binalizers,SequencyLen,BatchSize,Template51,Template51Profit,TimeOut=False,UseDOW=False,GenerateProfitArr=GenerateProfitArr)


In [29]:
WeightMatrix2=GetWeightMatrixFromDatagen(train_datagen,Step=3)
WeightMatrix2

{0: 3.617461519051224, 1: 0.4112538278936007, 2: 3.4249537119990445}

In [71]:
#Границы классификации для предварительной оценки доходности в калбеке
STDBorders=[0,0.5,0.7,1,1.5,2]
ChartTestList=[]
ChartValList=[]
EstimateCallback=MyCustomCallback(STDBorders,ChartTestList,ChartValList,EpochStep=1,BatchStep=2)
model_10=create_ResNet_model(train_datagen[0][0][0].shape,20,8,l2Conv=0.0005,OutActivation='softmax',OutNeurons=3)
model_10.compile(loss="categorical_crossentropy",optimizer=adamopt,metrics=['accuracy','mae'])
history1=model_10.fit(train_datagen,validation_data=test_datagen,callbacks=[EstimateCallback],class_weight=WeightMatrix2,epochs=10,verbose=1)#

Epoch 1/10
68/68 [==============================] - ETA: 0s - loss: 2.9989 - accuracy: 0.3215 - mae: 0.4474
Check std= [0, 0.5, 0.7, 1, 1.5, 2] P [12.5, 11.5, 11.3, 9.4, 7.2, 9.8] [19.7, 19.8, 19.6, 17.7, 15.9, 13.6] PM [16.1, 15.65, 15.5, 13.55, 11.6, 11.7] R [67.2, 47.0, 39.5, 29.9, 17.4, 9.3] [68.2, 47.3, 39.7, 30.2, 18.0, 9.8]
68/68 [==============================] - 293s 4s/step - loss: 2.9989 - accuracy: 0.3215 - mae: 0.4474 - val_loss: 2.6307 - val_accuracy: 0.0975 - val_mae: 0.4480
Epoch 2/10
68/68 [==============================] - ETA: 0s - loss: 2.5228 - accuracy: 0.3181 - mae: 0.4470
Check std= [0, 0.5, 0.7, 1, 1.5, 2] P [16.2, 15.9, 16.5, 16.3, 17.3, 17.5] [16.3, 16.1, 15.3, 14.1, 12.0, 12.2] PM [16.3, 16.05, 15.9, 15.25, 14.7, 14.85] R [70.4, 48.4, 40.5, 30.0, 16.7, 9.0] [71.7, 49.4, 41.5, 31.2, 18.4, 10.0]
68/68 [==============================] - 257s 4s/step - loss: 2.5228 - accuracy: 0.3181 - mae: 0.4470 - val_loss: 2.3790 - val_accuracy: 0.5011 - val_mae: 0.4415
Epoch

Check std= [0, 0.5, 0.7, 1, 1.5, 2] P [12.5, 11.5, 11.3, 9.4, 7.2, 9.8] [19.7, 19.8, 19.6, 17.7, 15.9, 13.6] PM [16.1, 15.65, 15.5, 13.55, 11.6, 11.7] R [67.2, 47.0, 39.5, 29.9, 17.4, 9.3] [68.2, 47.3, 39.7, 30.2, 18.0, 9.8]

Check std - Границы классификации

P - Доходность на тестовой и валидационной

PM - Средняя доходность тестовой и валидационной

R - Рекал

In [74]:
model_10.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 32, 9)]           0         
                                                                 
 dbl_104 (DBL)               (None, 32, 8)             248       
                                                                 
 res_block_10 (ResBlock)     (None, 16, 16)            1056      
                                                                 
 res_block_11 (ResBlock)     (None, 8, 32)             6144      
                                                                 
 res_block_12 (ResBlock)     (None, 4, 64)             75008     
                                                                 
 res_block_13 (ResBlock)     (None, 2, 128)            293376    
                                                                 
 res_block_14 (ResBlock)     (None, 1, 256)            629